# 생물정보학 및 실습 1 - Term Project

```
생물정보학 및 실습 1
서울대학교 협동과정 생물정보학전공 2022년 1학기

2021-20471 Myeongkyu Park

이 노트북은 Creative Commons BY-SA 조건으로 재사용 가능합니다.
장혜식 <hyeshik@snu.ac.kr>, 서울대학교 생명과학부
```




In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!git clone https://github.com/hyeshik/colab-biolab.git
!cd colab-biolab && bash tools/setup.sh
exec(open('colab-biolab/tools/activate_conda.py').read())

Cloning into 'colab-biolab'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 69 (delta 22), reused 54 (delta 13), pack-reused 0
Unpacking objects: 100% (69/69), done.
./
./root/
./root/.condarc
./root/.bin.priority/
./root/.bin.priority/pip2
./root/.bin.priority/pip
./root/.bin.priority/pip3
./root/.tmux.conf
./root/.profile
./root/.vimrc
./root/.bashrc.biolab
--2022-06-09 13:41:58--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76607678 (73M) [application/x-sh]
Saving to: ‘miniconda3.sh’

miniconda3.sh       100%[===================>]  73.06M   303MB/s    in 0.2s    

2022-06-09 13:41:59 (303 MB/s) - ‘miniconda3.sh’ save

In [3]:
!conda install -y bedtools bioawk

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



본 노트북에서는 레퍼런스 논문의 Figure 2E를 재현한다. 이 그림은 LIN28A와 상호작용 하는 서열 주위의 Watson-Crick (WC) pair co-occurence의 상대 빈도를 나타낸다.

LIN28A과 잘 알려진 target인 Pre-let-7이 상호작용할 때,  binding motif가 언제나 stem 근처의 terminal loop에 위치해 있으며, pre-let-7의 stem이 binding을 돕는다는 데이터가 있다. 따라서 binding motif를 새롭게 발견하면서 이런 현상이 유지되는지 확인하고자 그 주위의 2nd structure를 분석하는 것이 이 figure의 목적이 된다.

### Data Processing

먼저 논문과 같이 read depth>=50 조건으로 filtering 하기 위해서 CLIP-35L33G.bam 파일을 pileup 한다.

In [4]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!samtools mpileup CLIP-35L33G.bam > CLIP-35L33G.pileup

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


In [ ]:
!awk '$4 >= 50 { print $0; }' CLIP-35L33G.pileup > filtered-CLIP-35L33G.pileup

Pandas 패키지 이용해 만들어진 pileup 파일을 정리하고 match와 substituion만 남겨서 분석에 이용한다.

In [5]:
import pandas as pd
import re

pileup = pd.read_csv('filtered-CLIP-35L33G.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])

toremove = re.compile('[0-9]|[<>$*#-]|\^.|\+')
pileup['matches'] = pileup['basereads'].apply(lambda x: toremove.sub('', x))

In [7]:
pileup

,chrom,pos,_ref,count,basereads,quals,matches
0,chr1,3222722,N,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...,HCGGHEIHHHIBDDIIIGIG>GIHGDHHDDIAEIEEBIGIBGG8GI...,CCCAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
1,chr1,3222723,N,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,E>HIHFIIHHIGDFIII<IHBIIHGDHEB?I@HIEIBIII;>G;GI...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
2,chr1,3222724,N,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,H>GIGHHIHHDG:@IIIBIHDIIHG=HHDDG;IHHH;IIIDG?DIE...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
3,chr1,3222725,N,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,HGDEHHIIHHIG?DIIIEIGDIIHDBBH=DI;IIHBBIII4GD3GI...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...
4,chr1,3222726,N,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,HB:IHGIIGGIGDFHII@IFBHGHGBHGCBI@IIHGBIGI;>E@BI...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...
...,...,...,...,...,...,...,...
25210334,MU069435.1,1564,N,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,DIDHGHIIGIGIEDIGIHIIG?FBGG@E=:A7G8@HHIHIIDDBII...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
25210335,MU069435.1,1565,N,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,DIDHBHHIIIGICGIGIEIHDEFBGE8C=1?1G88IHIHIIDGBHI...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
25210336,MU069435.1,1566,N,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,DIDHG>IIHIHHCEIIIEB5D@;=H5BIHIHHI<DAIIBDIIFIIH...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
25210337,MU069435.1,1567,N,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,DIDHFHGIIIIIDDIII?B=G::B=G=DIHIHIID@BIIGGIIHIH...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...


In [8]:
filtered_pileup = pileup[pileup['matches'].str.len() >= 50]

In [8]:
filtered_pileup

,chrom,pos,_ref,count,basereads,quals,matches
0,chr1,3222722,N,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...,HCGGHEIHHHIBDDIIIGIG>GIHGDHHDDIAEIEEBIGIBGG8GI...,CCCAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
1,chr1,3222723,N,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,E>HIHFIIHHIGDFIII<IHBIIHGDHEB?I@HIEIBIII;>G;GI...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
2,chr1,3222724,N,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,H>GIGHHIHHDG:@IIIBIHDIIHG=HHDDG;IHHH;IIIDG?DIE...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
3,chr1,3222725,N,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,HGDEHHIIHHIG?DIIIEIGDIIHDBBH=DI;IIHBBIII4GD3GI...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...
4,chr1,3222726,N,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,HB:IHGIIGGIGDFHII@IFBHGHGBHGCBI@IIHGBIGI;>E@BI...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...
...,...,...,...,...,...,...,...
25210334,MU069435.1,1564,N,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,DIDHGHIIGIGIEDIGIHIIG?FBGG@E=:A7G8@HHIHIIDDBII...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
25210335,MU069435.1,1565,N,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,DIDHBHHIIIGICGIGIEIHDEFBGE8C=1?1G88IHIHIIDGBHI...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
25210336,MU069435.1,1566,N,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,DIDHG>IIHIHHCEIIIEB5D@;=H5BIHIHHI<DAIIBDIIFIIH...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
25210337,MU069435.1,1567,N,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,DIDHFHGIIIIIDDIII?B=G::B=G=DIHIHIID@BIIGGIIHIH...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...


Shannon entropy 계산하여 CRES>0.8 기준으로 filtering.
Shannon entropy 계산식은 레퍼런스 논문의 Supplementary part를 참조하였다.

In [9]:
import numpy as np

def CRES(seq):   
    counter = {}

    for base in ['A', 'T', 'G', 'C']:
        counter[base] = seq.count(base) # + seq.count(base.lower())

    shannon = 0
    for base in ['A', 'T', 'G', 'C']:
        if counter[base] > 0:
          p_n = counter[base] / len(seq)
          shannon += (p_n * np.log2(p_n))
        
    return -1 * shannon

In [10]:
filtered_pileup['CRES'] = filtered_pileup['matches'].apply(CRES)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [89]:
data = filtered_pileup[filtered_pileup.CRES >= 1.2].copy().reset_index().drop(['index', '_ref', 'count', 'basereads', 'quals'], axis=1)
# 여기서는 CRES cutoff 기준이 1.2로 되어있는데,
# 이는 이후 분석에서 noise가 발견되어 cutoff 기준을 강화한 것이다.
# Github README.md의 Google slide 참조

In [90]:
data

,chrom,pos,matches,CRES
0,chr1,3222726,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,1.401164
1,chr1,3224576,CCCGGGGGGGGGCGGGGGGGGGGGGGGGCTCTGCCCCCCCGCCCGG...,1.261116
2,chr1,4841534,AGAAGAAAAGGAAGAAAAAAAGAAAAGGATTTTTAAAAAAAAAAAA...,1.685160
3,chr1,4878699,AAGGGGGGGGGGGGGGGGCCAAGGAAGAGCCAAAACGAGAGGGGGG...,1.370870
4,chr1,4902568,AAAAAAAAAAAGGGGGAGAAAAGAAAGAGAGGGCGGGGGGGGGCCC...,1.277825
...,...,...,...,...
6071,chrM,11706,AAAAAAAAAAGGGGGGGGGGGGGGGGGGGgggggggggggGGGGGG...,1.268338
6072,chrM,15317,GGGGGTCTTTTTTCTTTGGTCCTTTTGTTCGTTTTTTGTGTTCTCT...,1.425942
6073,MU069435.1,386,TTGGGGGTGTGGTGTGTTGTTTTGGGGTCCCGTGTGCGCGGATTGC...,1.557113
6074,MU069435.1,581,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,1.314127


In [91]:
# 분석 중 지나치게 가까운 위치(20nt 이내)를 가리키는 row가 존재함을 발견.
# 이는 base frequency와 binding motif의 clarity에 영향을 주므로 제거하였다.
suspicious = []
for row in range(data.shape[0]):
  if row == 6075: break

  chr1 = data.iloc[row]['chrom']
  pos1 = data.iloc[row]['pos']
  
  chr2 = data.iloc[row+1]['chrom']
  pos2 = data.iloc[row+1]['pos']
  
  if chr1 == chr2:
    if pos2 - pos1 <= 20:
      suspicious.append((row, row+1))
remove = [x[1] for x in suspicious]
data1 = data.drop(remove)

In [40]:
# bedtools를 사용하여 sequence를 추출하기 위해서 bed 파일을 만들었다.
bed = data1[['chrom', 'pos']]
bed['st'] = bed['pos'] - 16
bed['ed'] = bed['pos'] + 15
bed = bed.drop(['pos'], axis=1)
bed.to_csv("bedfile.bed",header=None, index=False, sep='\t')

Sequence extract

In [43]:
!bedtools getfasta -fi GRCm39_fix.fasta -bed bedfile.bed -fo GRCm39.fasta.out
# chrM과 MU069435.1 염색체가 GRC39m reference genome에 포함되어 있지 않아 추출할 수 없었다.
# 단 그러한 경우가 6000여 개 중 5개에 불과하여 여기서는 무시하였다.

WARNING. chromosome (chrM) was not found in the FASTA file. Skipping.
WARNING. chromosome (chrM) was not found in the FASTA file. Skipping.
WARNING. chromosome (MU069435.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (MU069435.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (MU069435.1) was not found in the FASTA file. Skipping.


In [44]:
with open('GRCm39.fasta.out', 'r') as f1:
  with open('noheader_U.fasta.out', 'w') as f2:
    for l in f1:
      if l.startswith('>'): continue
      _l = l.replace('T', 'U')
      f2.write(_l)

In [48]:
with open('noheader_U.fasta.out', 'r') as f1:
  with open('AAGNHG.fasta.out', 'w') as f2:
    for l in f1:
      if l[13:16] == 'AAG' and l[17] != 'G' and l[18] == 'G':
        f2.write(l)

In [76]:
with open('noheader_U.fasta.out', 'r') as f1:
  with open('AAGNGH.fasta.out', 'w') as f2:
    for l in f1:
      if l[13:16] == 'AAG' and l[17] == 'G' and l[18] != 'G':
        f2.write(l)

Heatmap

In [54]:
pos_basecount = [{},{},{},{},{},{},{},{},{},{},
                 {},{},{},{},{},{},{},{},{},{},
                 {},{},{},{},{},{},{},{},{},{},{}]

reads = []
with open('AAGNHG.fasta.out', 'r') as f:
  for l in f:
    reads.append(l.strip())
    for i in range(len(l.strip())):
      store = pos_basecount[i]
      store.setdefault(l[i], 0)
      store[l[i]] += 1
      pos_basecount[i] = store

whole = sum(pos_basecount[0].values())
for pos in range(len(pos_basecount)):
  for base in ['A', 'C', 'G', 'U']:
    pos_basecount[pos].setdefault(base, 0)
  for k in pos_basecount[pos].keys():
    pos_basecount[pos][k] /= whole

In [60]:
Rmatrix = []
for pos1 in range(30):
  vector = []
  for pos2 in range(pos1+1, 31):
    vector.append(pos_basecount[pos1]['A'] * pos_basecount[pos2]['U'] 
                  + pos_basecount[pos1]['C'] * pos_basecount[pos2]['G']
                  + pos_basecount[pos1]['G'] * pos_basecount[pos2]['C']
                  + pos_basecount[pos1]['U'] * pos_basecount[pos2]['A'])
  Rmatrix.append(vector)

In [64]:
matcher = {'A': 'U', 'C': 'G', 'G': 'C', 'U': 'A'}
matrix = []
      
for pos1 in range(30):
  vector = []
  for pos2 in range(pos1+1, 31):
    match = 0
    for read in reads:
      base1 = read[pos1]
      base2 = read[pos2]

      if matcher[base1] == base2: match += 1

    vector.append(match / len(reads))

  matrix.append(vector)

In [72]:
compare = []
for i in range(len(matrix)):
  vector = []
  for j in range(len(matrix[i])):
    a = float(matrix[i][j])
    b = float(Rmatrix[i][j])
    if b == 0: vector.append(-1)
    else: vector.append(a/b)
  compare.append(vector)

In [73]:
for row in compare:
  print(['{:.4f}'.format(x) for x in row])

['0.6740', '0.9587', '1.1365', '1.0868', '0.9304', '0.9317', '1.0259', '1.0777', '1.0741', '1.0792', '1.1243', '1.0930', '1.0000', '1.0000', '1.0000', '0.9486', '0.9850', '1.0000', '1.0214', '0.9400', '1.0334', '1.0257', '1.0621', '1.0647', '0.8912', '1.0100', '0.9252', '0.9707', '1.0924', '1.1049']
['0.8265', '0.8982', '0.9902', '1.1710', '1.0009', '0.9941', '0.9834', '0.9056', '1.0011', '0.9078', '1.0519', '1.0000', '1.0000', '1.0000', '0.9625', '0.9710', '1.0000', '1.0502', '0.9458', '1.0126', '1.0191', '0.9405', '1.0678', '1.0191', '1.0996', '1.0029', '1.0914', '0.9911', '0.9970']
['0.6618', '0.9768', '0.9350', '1.0723', '0.8720', '1.0540', '0.9838', '0.9782', '1.0512', '1.1000', '1.0000', '1.0000', '1.0000', '1.0855', '1.0619', '1.0000', '0.9908', '1.0864', '1.0636', '1.0572', '0.9781', '1.0312', '1.0532', '1.0462', '0.9927', '0.9962', '0.9491', '0.9734']
['0.7024', '1.0240', '1.0573', '1.0830', '0.9671', '0.9404', '1.1791', '0.9739', '0.9778', '1.0000', '1.0000', '1.0000', '1.035

In [84]:
pos_basecount2 = [{},{},{},{},{},{},{},{},{},{},
                 {},{},{},{},{},{},{},{},{},{},
                 {},{},{},{},{},{},{},{},{},{},{}]

reads2 = []
with open('AAGNGH.fasta.out', 'r') as f:
  for l in f:
    reads2.append(l.strip())
    for i in range(len(l.strip())):
      store = pos_basecount2[i]
      store.setdefault(l[i], 0)
      store[l[i]] += 1
      pos_basecount2[i] = store

whole = sum(pos_basecount2[0].values())
for pos in range(len(pos_basecount2)):
  for base in ['A', 'C', 'G', 'U']:
    pos_basecount2[pos].setdefault(base, 0)
  for k in pos_basecount2[pos].keys():
    pos_basecount2[pos][k] /= whole

In [85]:
Rmatrix2 = []
for pos1 in range(30):
  vector = []
  for pos2 in range(pos1+1, 31):
    vector.append(pos_basecount2[pos1]['A'] * pos_basecount2[pos2]['U'] 
                  + pos_basecount2[pos1]['C'] * pos_basecount2[pos2]['G']
                  + pos_basecount2[pos1]['G'] * pos_basecount2[pos2]['C']
                  + pos_basecount2[pos1]['U'] * pos_basecount2[pos2]['A'])
  Rmatrix2.append(vector)

In [86]:
matrix2 = []
      
for pos1 in range(30):
  vector = []
  for pos2 in range(pos1+1, 31):
    match = 0
    for read in reads2:
      base1 = read[pos1]
      base2 = read[pos2]

      if matcher[base1] == base2: match += 1

    vector.append(match / len(reads2))

  matrix2.append(vector)

In [87]:
compare2 = []
for i in range(len(matrix2)):
  vector = []
  for j in range(len(matrix2[i])):
    a = float(matrix2[i][j])
    b = float(Rmatrix2[i][j])
    if b == 0: vector.append(-1)
    else: vector.append(a/b)
  compare2.append(vector)

In [88]:
for row in compare2:
  print(['{:.4f}'.format(x) for x in row])

['0.6425', '0.9549', '0.9044', '0.9613', '0.9460', '1.0004', '0.9100', '0.8762', '1.2226', '0.9865', '1.0912', '0.8532', '1.0000', '1.0000', '1.0000', '0.9318', '1.0000', '0.9426', '0.9773', '1.1363', '1.0654', '0.9086', '1.0135', '1.0953', '1.1299', '1.1326', '1.0024', '0.9081', '0.9625', '0.9299']
['0.7351', '1.0695', '0.9780', '1.0288', '1.1702', '0.9795', '0.9344', '1.0353', '1.0369', '0.9867', '0.9571', '1.0000', '1.0000', '1.0000', '0.8896', '1.0000', '1.0875', '0.9743', '1.1312', '0.9583', '0.9431', '1.0551', '1.0803', '1.0314', '0.9619', '1.0431', '1.1039', '1.0343', '1.0400']
['0.8331', '0.9219', '0.9789', '1.0026', '0.9038', '1.0935', '1.0006', '1.0523', '0.9296', '1.0299', '1.0000', '1.0000', '1.0000', '1.0773', '1.0000', '1.1697', '1.0989', '1.1550', '1.1188', '1.1581', '1.1239', '1.1517', '1.0674', '0.9504', '1.1168', '1.0337', '0.9683', '1.1346']
['0.7299', '0.8990', '1.0365', '1.1755', '0.9080', '1.1958', '1.0852', '1.0035', '0.8120', '1.0000', '1.0000', '1.0000', '0.875

Additional) Figure 2B

In [45]:
with open('noheader_U.fasta.out', 'r') as f:
  counter = {}
  for l in f:
    substring = l[13:19]
    counter.setdefault(substring, 0)
    counter[substring] += 1

In [112]:
sorted_dict = sorted(counter.items(), key = lambda item: item[1], reverse = True)

whole = sum(counter.values())
i = 0
for k, v in sorted_dict:
  i += 1
  print('{}\t{:.2f}%'.format(k, v/whole*100))
  if i >= 10: break

AAGGAG	3.28%
AAGAGG	2.10%
AAGAGA	1.90%
GAGGAG	1.86%
AAGGUG	1.53%
AAGAAG	1.41%
AAGAGC	1.16%
AAGAGU	1.13%
AAGUGG	1.05%
AAGCAG	0.98%


{'AAAGGA': 1,
 'AAAGGU': 1,
 'AAAGUG': 1,
 'AAAGUU': 1,
 'AAAUGG': 1,
 'AACAGG': 1,
 'AAGAAA': 21,
 'AAGAAC': 1,
 'AAGAAG': 85,
 'AAGAAU': 4,
 'AAGACA': 1,
 'AAGACC': 1,
 'AAGACG': 4,
 'AAGACU': 1,
 'AAGAGA': 114,
 'AAGAGC': 70,
 'AAGAGG': 126,
 'AAGAGU': 68,
 'AAGAUA': 2,
 'AAGAUC': 2,
 'AAGAUG': 51,
 'AAGAUU': 6,
 'AAGCAA': 6,
 'AAGCAC': 1,
 'AAGCAG': 59,
 'AAGCAU': 1,
 'AAGCCG': 3,
 'AAGCGA': 15,
 'AAGCGC': 11,
 'AAGCGG': 10,
 'AAGCGU': 12,
 'AAGCUG': 8,
 'AAGCUU': 1,
 'AAGGAA': 15,
 'AAGGAC': 5,
 'AAGGAG': 197,
 'AAGGAU': 24,
 'AAGGCA': 5,
 'AAGGCG': 29,
 'AAGGCU': 9,
 'AAGGGA': 53,
 'AAGGGC': 18,
 'AAGGGG': 28,
 'AAGGGU': 29,
 'AAGGUA': 41,
 'AAGGUC': 3,
 'AAGGUG': 92,
 'AAGGUU': 13,
 'AAGUAC': 1,
 'AAGUAG': 24,
 'AAGUCC': 1,
 'AAGUCG': 3,
 'AAGUGA': 56,
 'AAGUGC': 20,
 'AAGUGG': 63,
 'AAGUGU': 26,
 'AAGUUG': 10,
 'AAGUUU': 2,
 'ACAAAC': 1,
 'ACCGAG': 1,
 'ACGAAC': 1,
 'ACGAAG': 3,
 'ACGAAU': 1,
 'ACGACG': 2,
 'ACGAGA': 15,
 'ACGAGC': 3,
 'ACGAGG': 11,
 'ACGAGU': 9,
 'ACGAUG': 8,
